In [84]:
import pandas as pd
import geopandas as gpd
import os
from oauth2client.service_account import ServiceAccountCredentials
import gspread

## Opening Performance Data

In [85]:
#Authorize the API
scope = [
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/drive.file'
    ]
file_name = r'C:\Users\User\OneDrive - Migo (1)\Documents\Migo\Data Project\Masterfile\Credentials\update-mds-d3d8ff0bb675.json'
creds = ServiceAccountCredentials.from_json_keyfile_name(file_name,scope)
client = gspread.authorize(creds)

In [86]:
# get the instance of the Spreadsheet
sheet = client.open('Wargo Monthly Performance')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.get_worksheet(0)

# get all the records of the data
records_data = sheet_instance.get_all_records()
performance = pd.DataFrame.from_dict(records_data)
performance

,calendar_month_utc7,store_id,store_name,watchers_all_time_first,watchers_monthly_first,watchers_quarterly_first,gross_revenue
0,2022-04-01,C01152,Khumairah Cell,1109,1109,1109,41310000
1,2022-04-01,C01654,Warkop 29,1315,1459,1459,40826000
2,2022-04-01,C01066,Warteg Sederhana Pasar Perumnas,190,206,206,6180000
3,2022-04-01,C02043,Warung Mesta,407,558,558,18660000
4,2022-04-01,C01009,Kedai Rakyat Yu Tia,40,40,40,14150000
...,...,...,...,...,...,...,...
30502,2022-10-01,C01973,Arta Cell 6,0,1,1,0
30503,2023-02-01,C01792,AAN Cell,0,0,0,0
30504,2022-10-01,C02467,Warkop 995,0,0,0,0
30505,2023-01-01,C02354,Aqila Cell,0,0,0,0


In [87]:
#Pivot performance
performance_pivot = performance.pivot_table(index='store_id', values=['watchers_all_time_first','watchers_monthly_first','watchers_quarterly_first','gross_revenue'],
                                            aggfunc={'watchers_all_time_first':'sum',
                                                     'watchers_monthly_first':'sum',
                                                     'watchers_quarterly_first':'sum',
                                                     'gross_revenue':'sum'}).reset_index()

## Preprocessing

In [88]:
#Assigning Paths
path_adm = r'Z:\Masterfile\1. Shapefile\1. Population and Administrative Boundaries\3. New Area'
path_masterfile = r'C:\Users\User\OneDrive - Migo (1)\Documents\Migo\Data Project\Masterfile'
path_zmp = r'Z:\Masterfile\1. Shapefile\8. ZMP'
path_performance = r'Z:\Masterfile\3. Files\2. Performance Q4'

In [89]:
masterfile.sitting_type.unique()

array(['Coverage', 'Business Builder', 'BB Mitra', 'BB Ex-Host',
       'Association', 'Transportation', 'SRC'], dtype=object)

In [90]:
# Reading Files
region = gpd.read_file(os.path.join(path_adm,'new_area.gpkg'))
masterfile = pd.read_excel(os.path.join(path_masterfile, 'Store Score Masterfile.xlsx'), sheet_name='Masterfile New')
masterfile['area_lookup'] = masterfile['Area_Wilayah'].str.lower().str.replace(" ","").str.strip()
masterfile['sitting_type'] = masterfile.sitting_type.str.replace("Normal B", "Coverage").str.replace("Normal A","Coverage").str.replace("Normal","Coverage")
zmp = gpd.read_file(os.path.join(path_zmp, 'zmp_masterfile.gpkg'))
zmp = zmp[['ZMP_NAME', 'zmp_category']]
zmp['ZMP_NAME'] = zmp['ZMP_NAME'].str.lower().str.replace(" ","").str.strip()
performance_q4 = pd.read_excel(os.path.join(path_performance, 'Q4_Performance_alltochurned.xlsx'), sheet_name='All_Store')

In [91]:
#Checking columns
region.columns

Index(['OBJECTID', 'PROVINCE', 'CITY', 'KECA', 'PCK', 'SUM_AREA', 'SUM_POP',
       'SUM_DENSIT', 'Shape_Leng', 'Shape_Area', 'Area', 'geometry'],
      dtype='object')

### Join Masterfile with Region, Performance , and ZMP

In [92]:
performance_q4.columns

Index(['Store ID', 'Store Name', 'Service Center', 'City', 'Kecamatan',
       'Siting Type', 'Deployment Date', 'SME', 'Total Gross Revenue',
       'Total Unique Watchers', 'Count Normal Month', 'Count Watchlist Month',
       'Count Focus Month'],
      dtype='object')

In [93]:
# Join masterfile with region
region['new_id'] = region['CITY'].str.lower().str.strip() +" "+region['KECA'].str.lower().str.strip().str.replace(" ","")
masterfile['new_id'] = masterfile['City'].str.lower().str.strip() + " "+ masterfile['Kecamatan'].str.lower().str.strip().str.replace(" ","")
masterfile_join = masterfile.merge(region, how ='left', on = 'new_id').merge(performance_pivot, how ='left',left_on = 'Store_ID', right_on = 'store_id')\
.merge(zmp, how = 'left', left_on = 'area_lookup', right_on = 'ZMP_NAME').merge(performance_q4, how = 'left', left_on = 'Store_ID', right_on = 'Store ID')
masterfile_join = masterfile_join.rename(columns={
                                                      'Kecamatan_x':'Kecamatan', 'City_x' :'City', 'SUM_POP': 'population',
                                                      'status3':'status'})
masterfile_join.head()

,Area_Wilayah,Column1,Instruksi_Area,Nama_Toko,Jenis_Toko,Alamat,Lintang,Bujur,Kecamatan,City,...,City_y,Kecamatan_y,Siting Type,Deployment Date,SME,Total Gross Revenue,Total Unique Watchers,Count Normal Month,Count Watchlist Month,Count Focus Month
0,Jl. Jendral Sudirman Andir 2,NaN,Toko atau warung berada di sekitar area Jalan ...,ARTCELL 3,Toko Pulsa / Handphone,"Jl. Asia Afrika, RT:001RW:001, Kel.Kebon Jeruk...",-6.92092,107.6042,Andir,Kota Bandung,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jl. Jendral Sudirman Andir 2,NaN,Toko atau warung berada di sekitar area Jalan ...,ARTCELL 3,Toko Pulsa / Handphone,"Jl. Asia Afrika, RT:001RW:001, Kel.Kebon Jeruk...",-6.92092,107.6042,Andir,Kota Bandung,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jl. Rajawali Timur - Jl. Kebon Jati Andir,NaN,Toko berada di sekitar Jalan Rajawali Timur - ...,Bukit Barisan cell,Toko Pulsa / Handphone,"Jl. Stasiun Hall, RT:000RW:000, Kel.Kebon Jeru...",-6.87965,107.611813,Andir,Kota Bandung,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cirebon - Jungjang Arjawinangun,NaN,Toko atau warung berada di sekitar Ki Hajar De...,Nabil Cell,Toko Pulsa / Handphone,"Jalan Ki Hajar Dewantara Sebrang Rock Cell 2, ...",-6.64554,108.4103,Arjawinangun,Cirebon,...,Cirebon,Arjawinangun,coverage,9/14/2022,Wiwin Mulyana,0.0,10.0,1.0,1.0,1.0
4,Cirebon - Raya Pantura Cirebon Arjawinangun 3,NaN,Toko berada di sekitar Jalan Raya Pantura Cire...,Rudi Cell,Toko Pulsa / Handphone,Jalan Blok Tegallempuyang Tegalgubug Sebelah M...,-6.63424,108.388,Arjawinangun,Cirebon,...,Cirebon,Arjawinangun,coverage,9/14/2022,Wiwin Mulyana,0.0,9.0,1.0,2.0,0.0


In [94]:
#Preparing Masterfile to Update
masterfile_join = masterfile_join[['Area_Wilayah','Store_ID', 'Nama_Toko','Lintang', 'Bujur', 'Kecamatan', 'City', 'Province',
                         'status','new_id','sitting_type', 'population','gross_revenue', 'watchers_all_time_first','watchers_monthly_first','watchers_quarterly_first','zmp_category','Total Gross Revenue',
                         'Total Unique Watchers','Count Normal Month','Area']]
masterfile_join = masterfile_join.fillna(0)
masterfile_join = masterfile_join[masterfile_join['Store_ID']!=0]
masterfile_join = masterfile_join.fillna(0)

In [95]:
#Create Productivity
def category(x,a,b):
    if x[a] =='Aktif' and x[b]>0:
        return 'Productive Active'
    elif x[a]=='Aktif' and x[b]==0:
        return 'Not Productive Active'
    elif x[a]=='Churned' and x[b]>0:
        return 'Productive Churned'
    else:
        return 'Not Productive Churned'
masterfile_join['store_category'] = masterfile_join.apply(lambda x: category(x, 'status', 'Count Normal Month'), axis = 1)
masterfile_join.head()

,Area_Wilayah,Store_ID,Nama_Toko,Lintang,Bujur,Kecamatan,City,Province,status,new_id,...,gross_revenue,watchers_all_time_first,watchers_monthly_first,watchers_quarterly_first,zmp_category,Total Gross Revenue,Total Unique Watchers,Count Normal Month,Area,store_category
0,Jl. Jendral Sudirman Andir 2,C03000,ARTCELL 3,-6.92092,107.6042,Andir,Kota Bandung,Jawa Barat,Aktif,kota bandung andir,...,3000.0,4.0,5.0,5.0,Residential or Religion,0.0,0.0,0.0,BoBa,Not Productive Active
1,Jl. Jendral Sudirman Andir 2,C03000,ARTCELL 3,-6.92092,107.6042,Andir,Kota Bandung,Jawa Barat,Aktif,kota bandung andir,...,3000.0,4.0,5.0,5.0,Residential or Religion,0.0,0.0,0.0,BoBa,Not Productive Active
2,Jl. Rajawali Timur - Jl. Kebon Jati Andir,C03234,Bukit Barisan cell,-6.87965,107.611813,Andir,Kota Bandung,Jawa Barat,Aktif,kota bandung andir,...,10000.0,1.0,1.0,1.0,Residential or Religion,0.0,0.0,0.0,BoBa,Not Productive Active
3,Cirebon - Jungjang Arjawinangun,C02912,Nabil Cell,-6.64554,108.4103,Arjawinangun,Cirebon,Jawa Barat,Churned,cirebon arjawinangun,...,43000.0,6.0,8.0,7.0,0,0.0,10.0,1.0,CiMu,Productive Churned
4,Cirebon - Raya Pantura Cirebon Arjawinangun 3,C02928,Rudi Cell,-6.63424,108.388,Arjawinangun,Cirebon,Jawa Barat,Aktif,cirebon arjawinangun,...,15000.0,2.0,4.0,4.0,0,0.0,9.0,1.0,CiMu,Productive Active


# Updating to Google Sheets

In [96]:
#Reading from
spreadsheetId = "1P3mo9DnaZ9Kv3slU7AWSQCBuOJZgKPUMA5WAMhHXypY"
sheetId = "1821994154"

sh = client.open_by_key(spreadsheetId)
worksheet = sh.get_worksheet(2)
worksheet.clear()

{'spreadsheetId': '1P3mo9DnaZ9Kv3slU7AWSQCBuOJZgKPUMA5WAMhHXypY',
 'clearedRange': 'store_masterfile!A1:V13892'}

In [97]:
worksheet.update([masterfile_join.columns.values.tolist()] + masterfile_join.values.tolist())

{'spreadsheetId': '1P3mo9DnaZ9Kv3slU7AWSQCBuOJZgKPUMA5WAMhHXypY',
 'updatedRange': 'store_masterfile!A1:V2705',
 'updatedRows': 2705,
 'updatedColumns': 22,
 'updatedCells': 59510}

## Creating Shapefile

In [98]:
gdf[gdf.Store_ID =='C00709']

,Area_Wilayah,Store_ID,Nama_Toko,Lintang,Bujur,Kecamatan,City,Province,status,new_id,...,watchers_all_time_first,watchers_monthly_first,watchers_quarterly_first,zmp_category,Total Gross Revenue,Total Unique Watchers,Count Normal Month,Area,store_category,geometry


In [99]:
gdf = gpd.GeoDataFrame(masterfile_join, geometry=gpd.points_from_xy(masterfile_join.Bujur, masterfile_join.Lintang), crs = 4326)
gdf.to_file(r'Z:\Masterfile\1. Shapefile\9. Store Masterfile\store_masterfile.gpkg')